In [73]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import random

In [3]:
#Generates 1 random sequence
def generate_seq(length, features):
  return [np.random.randint(0, features-1) for _ in range(length)]

In [14]:
def oh_encode(sequence, features):
  encoding = []
  #Loop through each sequence (Go through each individual number in the sequence and OHE it)
  for value in sequence:
    #Give everything to be zero except for the index of the value itself
    vector = [0 for _ in range(features)]
    vector[value] = 1
    encoding.append(vector)
  return np.array(encoding)

In [24]:
#Takes the one hot encoded sequence and changes it back to the original sequence
def oh_decode(encoded_seq):
  return [np.argmax(value) for value in encoded_seq]

In [58]:
def generate_eg(length, n_features, out_idx):
  #Generate the sequence
  seq = generate_seq(length, n_features)
  enc = oh_encode(seq, n_features)
  #Convert from 2D to 3D, adding a dimension at axis=0 to represent 1 sample. Note that this does not change enc itself, but just returns a transformed array
  x = enc.reshape((1,length, n_features))
  #Get the one hot encoded array for a particular value in the sequence and add and extra dimension to it (The same as what you did for x)
  #You only apply this to one value (And not to all the 5 values in the sequence)
  y = enc[out_idx].reshape(1, n_features)
  return x, y

In [67]:
#One complete sequence will consist of 5 values (e.g [3,5,4,7,8] )
length = 5
#You need 10 numbers to represent one value (Since you OHE the values and there are a total of 10 different numbers, so each value is represented by 9 zeros and 1 one)
n_features = 10
#Get the y to be the 3rd value out of all the 5 values in one sequence
out_idx = 2

model = Sequential([
                    #Input shape to LSTM will be (length, n_features) where length is the length of one complete sequence (5 as each sequence has 5 values) and n_features is the no. of input features
                    #The first dimension is batch_size which is not defined as it can take on an arbitiary value (batch_size, length, n_features)
                    LSTM(25,input_shape=(length, n_features)),
                    #Softmax to predict among the n_features, which value to output as y
                    Dense(n_features, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 25)                3600      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                260       
Total params: 3,860
Trainable params: 3,860
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
for i in range(10000):
  x , y = generate_eg(length, n_features, out_idx)
  model.fit(x,y, epochs=1, verbose=0)

In [80]:
correct = 0
for i in range(100):
  x,y = generate_eg(length, n_features, out_idx)
  preds = model.predict(x)
  if oh_decode(preds) == oh_decode(y):
    correct += 1

print(f'Accuracy: {(correct/100) * 100}')

Accuracy: 100.0


In [82]:
from math import sin
from math import pi
from math import exp
from random import random
from random import randint
from random import uniform
from numpy import array
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
# generate damped sine wave in [0,1]
def generate_sequence(length, period, decay):
  return [0.5 + 0.5 * sin(2 * pi * i / period) * exp(-decay * i) for i in range(length)]

# generate input and output pairs of damped sine waves
def generate_examples(length, n_patterns, output):
  X, y = list(), list()
  for _ in range(n_patterns):
    p = randint(10, 20)
    d = uniform(0.01, 0.1)
    sequence = generate_sequence(length + output, p, d)
    X.append(sequence[:-output])
    y.append(sequence[-output:])
  X = array(X).reshape(n_patterns, length, 1)
  y = array(y).reshape(n_patterns, output)
  return X, y


In [83]:
x, y = generate_examples(50,10000,5)

In [88]:
x.shape

(10000, 50, 1)